#**DCGAN applying to Anime Style Face**

In this notebook we are going to implement the DCGAN architecture for the dataset of anime characters containing cropped faces.

## Setup

In [0]:
##############################
#  SETUP PYTHON ENVIRONMENT  #  
##############################

!pip install torch torchvision
conda update pytorch torchvision

In [12]:
!wget -c https://www.dropbox.com/s/a0ypfj5rvlxux7l/mangacharac.zip?dl=1 -O mangacharac.zip
!unzip mangacharac.zip

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
  inflating: mangacharac/37919_2012.jpg  
  inflating: mangacharac/37920_2012.jpg  
  inflating: mangacharac/37921_2012.jpg  
  inflating: mangacharac/37922_2012.jpg  
  inflating: mangacharac/37923_2012.jpg  
  inflating: mangacharac/37924_2012.jpg  
  inflating: mangacharac/37925_2012.jpg  
  inflating: mangacharac/37926_2012.jpg  
  inflating: mangacharac/37927_2012.jpg  
  inflating: mangacharac/37928_2012.jpg  
  inflating: mangacharac/37929_2012.jpg  
  inflating: mangacharac/37930_2012.jpg  
  inflating: mangacharac/37931_2012.jpg  
  inflating: mangacharac/37932_2012.jpg  
  inflating: mangacharac/37933_2012.jpg  
  inflating: mangacharac/37934_2012.jpg  
  inflating: mangacharac/37935_2012.jpg  
  inflating: mangacharac/37936_2012.jpg  
  inflating: mangacharac/37937_2012.jpg  
  inflating: mangacharac/37938_2012.jpg  
  inflating: mangacharac/37939_2012.jpg  
  inflating: mangacharac/37940_2012.jpg 

## Helper functions

In [0]:
import glob
import os
from PIL import Image

import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset

def init_normal_weights(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
        if hasattr(m, "bias") and m.bias is not None:
            torch.nn.init.constant_(m.bias.data, 0.0)
    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)

def print_models(G, D):
    
    """ Prints model information for the generators and discriminators. """
    
    print("                   G                   ")
    print("---------------------------------------")
    print(G)
    print("---------------------------------------")

    print("                   D                   ")
    print("---------------------------------------")
    print(D)
    print("---------------------------------------")
    
class LambdaLR:
    
    """ LambdaLR or LambdaLearningRate
        Allow us to decrease the learning rate from a specific epoch ('decay_start_epoch')
        This accelerates learning and become and allows to be more precise for larger epochs.
    """
    
    def __init__(self, n_epochs, offset, decay_start_epoch):
        self.n_epochs = n_epochs
        self.offset = offset
        self.decay_start_epoch = decay_start_epoch
        
    def step(self, epoch):
        return 1.0 - max(0, epoch + self.offset - self.decay_start_epoch) / (self.n_epochs - self.decay_start_epoch)
    
class ImageDataset(Dataset):
    
    """ ImageDataSet
        Allow us to preprocess/transform data into a desired format (resize, crop, normalize, rgb, tensor)
    """
    
    def __init__(self, root, transforms_):
        self.transform = transforms.Compose(transforms_)
        self.files = sorted(glob.glob(os.path.join(root) + "/*.*"))

    def __getitem__(self, index):
        image = Image.open(self.files[index % len(self.files)])
        image = self.transform(image)
        return image
    
    def __len__(self):
        return len(self.files)

## Architectures

In [0]:
import torch.nn as nn

class Generator(nn.Module):

    """Defines the architecture of the generator network."""
    
    def __init__(self, opt):
        super(Generator, self).__init__()

        self.init_size = opt.img_size // 4
        self.linear = nn.Sequential(nn.Linear(opt.latent_dim, 128 * self.init_size ** 2))
        
        out_features = 128
        model = [nn.BatchNorm2d(out_features)]
        in_features = out_features
        for _ in range(2):
            out_features //= 2
            model += [
                    nn.Upsample(scale_factor = 2),
                    nn.Conv2d(in_features, out_features, 3, stride = 1, padding = 1),
                    nn.BatchNorm2d(out_features, 0.8),
                    nn.LeakyReLU(0.2, inplace = True)
                    ]
            in_features = out_features
        model += [nn.Conv2d(out_features, 3, opt.channels, stride = 1, padding = 1), nn.Tanh()]
        
        self.model = nn.Sequential(*model)

    def forward(self, x):
        x = self.linear(x)
        x = x.view(x.shape[0], 128, self.init_size, self.init_size)
        x = self.model(x)
        return x

class Discriminator(nn.Module):
    
    """Defines the architecture of the discriminator network."""
    
    def __init__(self, opt):
        super(Discriminator, self).__init__()

        def discriminator_block(in_filters, out_filters, batch_normalize = True):
            block = [nn.Conv2d(in_filters, out_filters, 3, stride = 2, padding = 1), nn.LeakyReLU(0.2, inplace=True), nn.Dropout2d(0.25)]
            if batch_normalize:
                block.append(nn.BatchNorm2d(out_filters, 0.8))
            return block

        self.model = nn.Sequential(
            *discriminator_block(opt.channels, 16, batch_normalize = False),
            *discriminator_block(16, 32),
            *discriminator_block(32, 64),
            *discriminator_block(64, 128),
        )

        # The height and width of downsampled image
        ds_size = opt.img_size // 4 ** 2
        self.output_layer = nn.Sequential(nn.Linear(128 * ds_size ** 2, 1), nn.Sigmoid())

    def forward(self, x):
        x = self.model(x)
        x = x.view(x.shape[0], -1)
        x = self.output_layer(x)
        return x

## DCGAN

In [18]:
import os
import sys
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
import argparse

import torch
import torchvision.transforms as transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader
from torch.autograd import Variable
from torchvision import datasets

##############################
#    MODEL CONFIGURATION     # 
##############################

parser = argparse.ArgumentParser()
# data set
parser.add_argument("--dataset_name", type=str, default="mangacharac", help="name of the dataset")
parser.add_argument("--img_size", type=int, default=64, help="size of each image dimension")
parser.add_argument("--channels", type=int, default=3, help="number of image channels")
# DCGAN parameters
parser.add_argument("--lr", type=float, default=0.0002, help="adam: learning rate")
parser.add_argument("--b1", type=float, default=0.3, help="adam: decay of first order momentum of gradient")
parser.add_argument("--b2", type=float, default=0.999, help="adam: decay of first order momentum of gradient")
parser.add_argument("--decay_epoch", type=int, default=15, help="epoch from which to start lr decay")
parser.add_argument("--latent_dim", type=int, default=100, help="dimensionality of the latent space")
# training parameters
parser.add_argument("--cuda", type=bool, default=False, help="change to GPU mode")
parser.add_argument("--n_epochs", type=int, default=1000, help = "number of epochs of training")
parser.add_argument("--batch_size", type=int, default=32, help="size of the batches")
parser.add_argument("--n_cpu", type=int, default=0, help="number of cpu threads to use during batch generation") # 0 by default in windows because multiprocessing doesn't work
# saving parameters
parser.add_argument("--epoch", type=int, default=0, help = "epoch to start training from")
parser.add_argument("--sample_interval", type=int, default=1000, help="interval between saving generator outputs")
parser.add_argument("--checkpoint_interval", type=int, default=1, help="interval between saving model checkpoints")
opt = parser.parse_args("")

# GPU option
cuda = torch.cuda.is_available()
if cuda:
    print('Models moved to GPU.')
    opt.cuda = True

# Create sample image, checkpoint and losses directories
os.makedirs("images", exist_ok = True)
os.makedirs("saved_models", exist_ok = True)
os.makedirs("losses_models", exist_ok = True)

# Configure dataloader
transforms_= [transforms.Resize(opt.img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]

dataloader = DataLoader(
    ImageDataset(opt.dataset_name, transforms_ = transforms_),
    batch_size = opt.batch_size,
    shuffle = True,
    num_workers = opt.n_cpu
)

##############################
#    MODEL INITIALIZATION    # 
##############################

def create_model(opt):
    
    """ Builds the generators and discriminators. """
    
    # Initialize generator and discriminator
    G = Generator(opt)
    D = Discriminator(opt)
    
    print_models(G, D)
    
    if opt.cuda:
        G = G.cuda()
        D = D.cuda()
        
    if opt.epoch != 0:
        # Load pretrained models
        G.load_state_dict(torch.load("saved_models/G_%d.pth" % opt.epoch))
        D.load_state_dict(torch.load("saved_models/D_%d.pth" % opt.epoch))
    else:
        # Initialize weights
        G.apply(init_normal_weights)
        D.apply(init_normal_weights)
        
    return G, D

##############################
#       MODEL TRAINING       # 
##############################
    
def training_loop(dataloader, opt):
    
    """ Runs the training loop.
        * Saves checkpoint every opt.checkpoint_interval iterations
        * Saves generated samples every opt.sample_interval iterations
    """
    
    # Create generators and discriminators
    G, D = create_model(opt)
    
    # Loss
    loss_GAN = torch.nn.BCELoss()

    # Optimizers
    optimizer_G = torch.optim.Adam(G.parameters(), lr = opt.lr, betas = (opt.b1, opt.b2))
    optimizer_D = torch.optim.Adam(D.parameters(), lr = opt.lr, betas = (opt.b1, opt.b2))

    # Learning rate update schedulers
    LambdaLR_schedular_G = torch.optim.lr_scheduler.LambdaLR(optimizer_G, lr_lambda = LambdaLR(opt.n_epochs, opt.epoch, opt.decay_epoch).step)
    LambdaLR_scheduler_D = torch.optim.lr_scheduler.LambdaLR(optimizer_D, lr_lambda = LambdaLR(opt.n_epochs, opt.epoch, opt.decay_epoch).step)
    
    Tensor = torch.Tensor
    if opt.cuda:  
        loss_GAN.cuda()
        Tensor = torch.cuda.FloatTensor

    losses_models_G = pd.DataFrame(np.zeros((len(dataloader), opt.n_epochs - opt.epoch + 1)))
    losses_models_D = pd.DataFrame(np.zeros((len(dataloader), opt.n_epochs - opt.epoch + 1)))
    losses_models_G.columns = range(opt.epoch, opt.n_epochs+1)
    losses_models_D.columns = range(opt.epoch, opt.n_epochs+1)
    # Training
    for epoch in range(opt.epoch, opt.n_epochs):
        for i, batch in enumerate(dataloader):
            
            # Configure input
            real_imgs = Variable(batch.type(Tensor))
            
            # Adversarial ground truths
            valid = Variable(Tensor(batch.shape[0], 1).fill_(1.0), requires_grad=False)
            fake = Variable(Tensor(batch.shape[0], 1).fill_(0.0), requires_grad=False)
            
            # ------------------
            #  Train Generator 
            # ------------------

            optimizer_G.zero_grad()
            
            # Sample noise as generator input
            z = Variable(Tensor(np.random.normal(0, 1, (batch.shape[0], opt.latent_dim))))

            # Generate a batch of images
            gen_imgs = G(z)

            # Generator loss
            loss_G = loss_GAN(D(gen_imgs), valid)
    
            loss_G.backward()
            optimizer_G.step()
            
            losses_models_G[epoch][i] = loss_G
                        
            # --------------------
            #  Train Discriminator 
            # --------------------
            
            optimizer_D.zero_grad()

            # Discriminator loss
            real_loss = loss_GAN(D(real_imgs), valid)
            fake_loss = loss_GAN(D(gen_imgs.detach()), fake)
            loss_D = (real_loss + fake_loss) / 2
    
            loss_D.backward()
            optimizer_D.step()
            
            losses_models_D[epoch][i] = loss_D
           
            # --------------
            #  Log Progress
            # --------------
        
            batches_done = epoch * len(dataloader) + i
        
            # Print log
            sys.stdout.write(
                "\r[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
                % (
                    epoch,
                    opt.n_epochs,
                    i,
                    len(dataloader),
                    loss_D.item(),
                    loss_G.item()
                )
            )
        
            # Save sample image at interval
            if batches_done % opt.sample_interval == 0:
                save_image(gen_imgs.data[:32], "images/%d.png" % batches_done, nrow=4, normalize=True)
        
        # Update learning rates
        LambdaLR_schedular_G.step()
        LambdaLR_scheduler_D.step()
        
        # Save discriminators and generators lossses at each epoch
        losses_models_G.to_pickle("losses_models/losses_models_G_%d" % epoch)
        losses_models_D.to_pickle("losses_models/losses_models_D_%d" % epoch)
            
        # Save model at checkpoints
        if opt.checkpoint_interval != -1 and epoch % opt.checkpoint_interval == 0:
            torch.save(G.state_dict(), "saved_models/G_%d.pth" % epoch)
            torch.save(D.state_dict(), "saved_models/D_%d.pth" % epoch)

Models moved to GPU.


In [19]:
training_loop(dataloader, opt)

                   G                   
---------------------------------------
Generator(
  (linear): Sequential(
    (0): Linear(in_features=100, out_features=32768, bias=True)
  )
  (model): Sequential(
    (0): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Upsample(scale_factor=2.0, mode=nearest)
    (2): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): BatchNorm2d(64, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Upsample(scale_factor=2.0, mode=nearest)
    (6): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): BatchNorm2d(32, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (8): LeakyReLU(negative_slope=0.2, inplace=True)
    (9): Conv2d(32, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): Tanh()
  )
)
---------------------------------------
                   D                

KeyboardInterrupt: ignored